In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from decimal import Decimal
from datetime import datetime
import re

%matplotlib inline

import findspark
findspark.init()

from pyspark.sql import *
from pyspark.sql.functions import  isnan, when, count, col,isnull

%matplotlib inline

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
data_folder = './data/'

In [28]:
data = spark.read.csv(data_folder+'en.openfoodfacts.org.products.csv',header = True, sep="\t")

N_subset = 0

if N_subset>0:
    print("Subset with", N_subset, "products of the csv taken.")
    rdd = spark.sparkContext.parallelize(data.take(N_subset))
    data = spark.createDataFrame(rdd, data.schema)

In [29]:
palm = data.select(['countries_en','ingredients_from_palm_oil_n','ingredients_that_may_be_from_palm_oil_n'])

In [30]:
palm = palm.na.drop()

In [31]:
palm_flat = palm.rdd.flatMap(lambda r:[(x, r['ingredients_from_palm_oil_n'], r['ingredients_that_may_be_from_palm_oil_n']) 
                                                for x in r['countries_en'].split(',')]).toDF()

In [32]:
palm_flat = palm_flat.selectExpr("_1 as countries_en", 
                                 "_2 as ingredients_from_palm_oil_n", 
                                 "_3 as ingredients_that_may_be_from_palm_oil_n")

In [33]:
palm_reduced = palm_flat.rdd.filter(lambda r: re.match(r'United State|France', r['countries_en'])).toDF()

In [34]:
palm_positive = palm_reduced.filter("ingredients_from_palm_oil_n>0 OR ingredients_that_may_be_from_palm_oil_n>0")

In [35]:
palm_positive.groupBy("countries_en").count().collect()

Py4JJavaError: An error occurred while calling o391.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 20.0 failed 1 times, most recent failure: Lost task 1.0 in stage 20.0 (TID 428, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "D:\Documents\etudes\epfl\cours\Applied_Data_Analysis\spark\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 253, in main
  File "D:\Documents\etudes\epfl\cours\Applied_Data_Analysis\spark\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 248, in process
  File "D:\Documents\etudes\epfl\cours\Applied_Data_Analysis\spark\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 379, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "D:\Documents\etudes\epfl\cours\Applied_Data_Analysis\spark\spark-2.3.2-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\seriali